# Análise de Cluster Hierárquico na Fluencypass
Este notebook reproduz passo a passo todo o fluxo de limpeza, normalização, clusterização hierárquica, visualizações e projeção PCA, conforme descrito no artigo.

## 1. Importação de Bibliotecas

In [ ]:
import pandas as pd
from sklearn.preprocessing import StandardScaler
from scipy.cluster.hierarchy import linkage, dendrogram, fcluster
from scipy.spatial.distance import pdist
from sklearn.decomposition import PCA
import matplotlib.pyplot as plt


## 2. Carregar Dados

In [ ]:
# Ajuste o caminho se necessário
df = pd.read_csv('fluencypass_students.csv', sep=';')
df.columns = ['total_activities', 'private_talks', 'group_talks']
df.head()


## 3. Limpeza e Verificação Inicial

In [ ]:
# Verificar informações básicas e estatísticas
print(df.info())
print(df.describe())


## 4. Normalização dos Dados

In [ ]:
scaler = StandardScaler()
dados_padronizados = scaler.fit_transform(df[['total_activities','private_talks','group_talks']])


## 5. Cálculo da Matriz de Distâncias e Linkage (Ward)

In [ ]:
distance_matrix = pdist(dados_padronizados, metric='euclidean')
linked = linkage(distance_matrix, method='ward')


## 6. Visualização do Dendrograma

In [ ]:
plt.figure(figsize=(12, 6))
dendrogram(linked, truncate_mode='lastp', p=30,
           leaf_rotation=45., leaf_font_size=10., show_contracted=True)
plt.title('Figura 1. Dendrograma da Análise de Cluster Hierárquico')
plt.xlabel('Amostras de Estudantes')
plt.ylabel('Distância Euclidiana')
plt.tight_layout()
plt.savefig('dendrograma_cluster.png')
plt.show()


## 7. Definição de 5 Clusters

In [ ]:
cluster_labels = fcluster(linked, t=5, criterion='maxclust')
df['cluster'] = cluster_labels
df['cluster'].value_counts().sort_index()


## 8. Cálculo do Perfil Médio por Cluster

In [ ]:
cluster_profile = df.groupby('cluster')[['total_activities','private_talks','group_talks']].mean().round(2)
cluster_profile


## 9. Redução de Dimensionalidade por PCA

In [ ]:
pca = PCA(n_components=2)
coords = pca.fit_transform(dados_padronizados)
print('Explained variance ratio:', pca.explained_variance_ratio_)


## 10. Plotagem da Projeção PCA com Clusters

In [ ]:
plt.figure(figsize=(8, 6))
plt.scatter(coords[:, 0], coords[:, 1], c=cluster_labels, cmap='tab10')
plt.title('Figura 2. Projeção dos Clusters em Espaço PCA')
plt.xlabel('Componente Principal 1')
plt.ylabel('Componente Principal 2')
plt.colorbar(label='Cluster')
plt.tight_layout()
plt.savefig('clusters_pca.png')
plt.show()


## 11. Exportação dos Resultados

In [ ]:
# Salvar CSVs de saída
df.to_csv('students_with_clusters.csv', index=False)
cluster_profile.to_csv('cluster_profiles.csv')
print('Arquivos gerados: dendrograma_cluster.png, clusters_pca.png, students_with_clusters.csv, cluster_profiles.csv')
